Introduction to integration in Python
=====================================

# Estimating the volume of a solid



We can use integrals to compute the volume of solids. If we know how the cross-sectional area of a solid varies in some direction, we simply evaluate the following integral:

$\int_{x0}^{x1} A(x) dx$

For a sphere, we can derive:

$A(x) = \pi (1 - x^2)$



In [3]:
import numpy as np

R = 1
x = np.linspace(-R, R, 1500)
A = np.pi * (1 - x**2)

approx_V = np.trapz(A, x)
exact_V = 4 / 3 * np.pi * R**3

print(f'''Approximate volume = {approx_V:1.4f}
Exact volume = {exact_V:1.4f}''')

Approximate volume = 4.1888
Exact volume = 4.1888


With 50 points, the estimate is pretty good. Try increasing the number of points to improve the estimate.



# Plotly

In [2]:
# x and y given as array_like objects
import plotly.express as px
fig = px.scatter(x=[0, 1, 2, 3, 4], y=[0, 1, 4, 9, 16])
fig.show()

ModuleNotFoundError: No module named 'plotly'

In [1]:
#  ___________________________________________________________________________
#
#  Pyomo: Python Optimization Modeling Objects
#  Copyright (c) 2008-2022
#  National Technology and Engineering Solutions of Sandia, LLC
#  Under the terms of Contract DE-NA0003525 with National Technology and
#  Engineering Solutions of Sandia, LLC, the U.S. Government retains certain
#  rights in this software.
#  This software is distributed under the 3-clause BSD License.
#  ___________________________________________________________________________
"""
Continuously stirred tank reactor model, based on
pyomo/examples/doc/pyomobook/nonlinear-ch/react_design/ReactorDesign.py
"""
import pandas as pd
from pyomo.environ import (
    ConcreteModel, Var, PositiveReals, Objective, Constraint, maximize,
    SolverFactory
)

def reactor_design_model(data):
    
    # Create the concrete model
    model = ConcreteModel()
    
    # Rate constants
    model.k1 = Var(initialize = 5.0/6.0, within=PositiveReals) # min^-1
    model.k2 = Var(initialize = 5.0/3.0, within=PositiveReals) # min^-1
    model.k3 = Var(initialize = 1.0/6000.0, within=PositiveReals) # m^3/(gmol min)
    model.k1.fixed = True
    model.k2.fixed = True
    model.k3.fixed = True
    
    # Inlet concentration of A, gmol/m^3
    model.caf = Var(initialize = float(data['caf']), within=PositiveReals)
    model.caf.fixed = True
    
	# Space velocity (flowrate/volume)
    model.sv = Var(initialize = float(data['sv']), within=PositiveReals)
    model.sv.fixed = True
    
    # Outlet concentration of each component
    model.ca = Var(initialize = 5000.0, within=PositiveReals) 
    model.cb = Var(initialize = 2000.0, within=PositiveReals) 
    model.cc = Var(initialize = 2000.0, within=PositiveReals) 
    model.cd = Var(initialize = 1000.0, within=PositiveReals)
    
    # Objective
    model.obj = Objective(expr = model.cb, sense=maximize)
    
    # Constraints
    model.ca_bal = Constraint(expr = (0 == model.sv * model.caf \
                     - model.sv * model.ca - model.k1 * model.ca \
                     -  2.0 * model.k3 * model.ca ** 2.0))
    
    model.cb_bal = Constraint(expr=(0 == -model.sv * model.cb \
                     + model.k1 * model.ca - model.k2 * model.cb))
    
    model.cc_bal = Constraint(expr=(0 == -model.sv * model.cc \
                     + model.k2 * model.cb))
    
    model.cd_bal = Constraint(expr=(0 == -model.sv * model.cd \
                     + model.k3 * model.ca ** 2.0))
    
    return model

def main():
    # For a range of sv values, return ca, cb, cc, and cd
    results = []
    sv_values = [1.0 + v * 0.05 for v in range(1, 20)]
    caf = 10000
    for sv in sv_values:
        model = reactor_design_model({'caf': caf, 'sv': sv})
        solver = SolverFactory('ipopt')
        solver.solve(model)
        results.append([sv, caf, model.ca(), model.cb(), model.cc(), model.cd()])
    
    results = pd.DataFrame(results, columns=['sv', 'caf', 'ca', 'cb', 'cc', 'cd'])
    print(results)
    

main()

      sv    caf           ca           cb           cc           cd
0   1.05  10000  3458.360964  1060.846921  1683.884002  1898.454056
1   1.10  10000  3535.060217  1064.777174  1613.298748  1893.431930
2   1.15  10000  3609.119151  1067.786731  1547.517002  1887.788558
3   1.20  10000  3680.713187  1069.974764  1486.076060  1881.617995
4   1.25  10000  3750.000000  1071.428571  1428.571429  1875.000000
5   1.30  10000  3817.121917  1072.225258  1374.647766  1868.002530
6   1.35  10000  3882.207905  1072.433123  1323.991510  1860.683731
7   1.40  10000  3945.375247  1072.112839  1276.324808  1853.093553
8   1.45  10000  4006.730956  1071.318437  1231.400503  1845.275052
9   1.50  10000  4066.372975  1070.098151  1188.997946  1837.265464
10  1.55  10000  4124.391210  1068.495132  1148.919497  1829.097081
11  1.60  10000  4180.868407  1066.548063  1110.987566  1820.797983
12  1.65  10000  4235.880919  1064.291688  1075.042109  1812.392642
13  1.70  10000  4289.499366  1061.757269  1040.